# Potato Nematode Predictor
This work contains the public sector consultancy work on a potato nematode predictor carried out by Aarhus University.

Start by configuring the notebook:

In [2]:
import wget
import geopandas
import os
import rasterio
import sys
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from zipfile import ZipFile
from tqdm.autonotebook import tqdm

from utils import RasterstatsMultiProc

# Automatically prints execution time for the individual cells
%load_ext autotime

# Automatically reloads functions defined in external files
%load_ext autoreload
%autoreload 2

# Set xarray to use html as display_style
xr.set_options(display_style="html")

# The path to the project (so absoute file paths can be used throughout the notebook)
PROJ_PATH = Path.cwd().parent

# Define which field polygons should be used for analysis (2017 to 2019 seem to follow the same metadata format)
FIELD_POLYGONS = ['FieldPolygons2017', 'FieldPolygons2018', 'FieldPolygons2019']

# Define global flags
MULTI_PROC_ZONAL_STATS = False
ALL_TOUCHED = False

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 29.6 ms


---
Download the field polygons from The Danish Agricultural Agency:

In [3]:
# Downloaded files will go into the 'data/external' folder
dest_folder = PROJ_PATH / 'data' / 'external'
if not dest_folder.exists():
    os.makedirs(dest_folder)
    
# Define the download links for the field polygons for the individual years
file_url_mapping = {
    'FieldPolygons2016.zip': 'https://kortdata.fvm.dk/download/DownloadStream?id=3037da0f2744a85adc8b08ca5c31c3cb',
    'FieldPolygons2017.zip': 'https://kortdata.fvm.dk/download/DownloadStream?id=d0c8946763e465bf9f6160a6bc40531f',
    'FieldPolygons2018.zip': 'https://kortdata.fvm.dk/download/DownloadStream?id=cfb1b47130b7276f8515fbaae60bde2a',
    'FieldPolygons2019.zip': 'https://kortdata.fvm.dk/download/DownloadStream?id=3d19613ac986ed05a7c301319738e332'
}

# Download the zipfiles
for filename, url in file_url_mapping.items():
    dest_path = PROJ_PATH / 'data' / 'external' / filename
    if not dest_path.exists():
        wget.download(url, str(dest_path))
        print("File has been downloaded: " + filename)
    else:
        print("File already exists: " + str(PROJ_PATH / 'data' / 'external' / filename))

File already exists: /home/jovyan/work/data/external/FieldPolygons2016.zip
File already exists: /home/jovyan/work/data/external/FieldPolygons2017.zip
File already exists: /home/jovyan/work/data/external/FieldPolygons2018.zip
File already exists: /home/jovyan/work/data/external/FieldPolygons2019.zip
time: 28.4 ms


---
Then extract the zipfiles:

In [4]:
# The extracted zipfiles will go into the 'data/raw' folder
for zipfile in (PROJ_PATH / 'data' / 'external').glob('**/*.zip'):
    dest_folder = PROJ_PATH / 'data' / 'raw' / zipfile.stem   
    if not dest_folder.exists():
        with ZipFile(str(zipfile), 'r') as zipObj:
            zipObj.extractall(str(dest_folder))
        print("Zipfile has been extracted: " + str(zipfile))
    else:
        print("Zipfile has already been extracted: " + str(zipfile))

Zipfile has already been extracted: /home/jovyan/work/data/external/FieldPolygons2019.zip
Zipfile has already been extracted: /home/jovyan/work/data/external/FieldPolygons2016.zip
Zipfile has already been extracted: /home/jovyan/work/data/external/FieldPolygons2017.zip
Zipfile has already been extracted: /home/jovyan/work/data/external/FieldPolygons2018.zip
time: 29.8 ms


---
Now load the shapefiles into geopandas dataframes:

In [5]:
def load_shp(shp_name):
    # Load shapefile into dataframe and remove NaN rows
    shp_file_path = list((PROJ_PATH / 'data' / 'raw' / shp_name).glob('**/*.shp'))[0]
    df = geopandas.read_file(str(shp_file_path))
    df = df.dropna()
    
    # Change all column names to be lower-case to make the naming consistent across years (https://stackoverflow.com/a/36362607/12045808)
    df.columns = map(str.lower, df.columns)
    
    return df

# Load the dataframes into a dict, with each year as a key
df_all = {}
for df_name in FIELD_POLYGONS:
    df = load_shp(df_name)
    df_all[df_name] = df

time: 1min 25s


---
Find the potato fields and count the number of unique sorts:

In [6]:
def extract_potato_fields(df):
    # Create a new dataframe with all the different types of potatoes
    df = df[df['afgroede'].str.contains("kartof", case=False)]  

    # Find the different potato types, count the number of fields for each type, and calculate total area for each type
    for potato_type in sorted(df['afgroede'].unique()):
        num_fields = df[df['afgroede'] == potato_type].shape[0]
        sum_area = df[df['afgroede'] == potato_type]['imk_areal'].sum()
        print("There are " + str(num_fields) + " fields (total area = " + str(int(sum_area)) + " ha) of type: " + potato_type)
        
    return df 

# Extract the potato fields and load them into a new dict with each year as a key
df_potato = {}
for df_name, df in df_all.items():
    print("### Analyzing " + df_name + " ###")
    df_potato[df_name] = extract_potato_fields(df)
    print("")

### Analyzing FieldPolygons2017 ###
There are 618 fields (total area = 4773 ha) of type: Kartofler, andre
There are 88 fields (total area = 586 ha) of type: Kartofler, lægge- (certificerede)
There are 1089 fields (total area = 6626 ha) of type: Kartofler, lægge- (egen opformering)
There are 2597 fields (total area = 9223 ha) of type: Kartofler, spise-
There are 3885 fields (total area = 28040 ha) of type: Kartofler, stivelses-

### Analyzing FieldPolygons2018 ###
There are 497 fields (total area = 3693 ha) of type: Kartofler, andre
There are 317 fields (total area = 2988 ha) of type: Kartofler, lægge- (certificerede)
There are 704 fields (total area = 3700 ha) of type: Kartofler, lægge- (egen opformering)
There are 2137 fields (total area = 7805 ha) of type: Kartofler, spise-
There are 3484 fields (total area = 26235 ha) of type: Kartofler, stivelses-

### Analyzing FieldPolygons2019 ###
There are 588 fields (total area = 4877 ha) of type: Kartofler, andre
There are 465 fields (total a

---
Calculate zonal statistics for the the potato fields for the different radar data measurements:

In [7]:
tif = list((PROJ_PATH / 'data' / 'raw' / 'Sentinel-1').glob('*.tif'))[0]
with rasterio.open(tif) as src:
    tif_crs = src.crs
    print("Projection used is: " + str(tif_crs))

for df_name, df in df_potato.items():
    # Set the CRS in the geodataframe to be wkt format (otherwise you won't be able to save as a shapefile)
    df_potato[df_name] = df_potato[df_name].to_crs({'init': tif_crs})

Projection used is: EPSG:4326


/opt/conda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


time: 2.58 s


In [8]:
'''
tifs = sorted((PROJ_PATH / 'data' / 'raw' / 'Sentinel-1').glob('*.tif'))
df_potato_stats = df_potato.copy()

for df_name, df in df_potato.items(): # Loop over all field polygon years
    pkl_name = df_name + '_stats' 
    pkl_path = (PROJ_PATH / 'data' / 'processed' / pkl_name).with_suffix('.pkl')
    if pkl_path.exists():
        print("Zonal statistics have already been calculated for: " + df_name)
    else:
        print("Calculating zonal statistics for: " + df_name)
        #df = df.head(20)  # For debugging to (ie. only process 20 fields)
        for tif in tqdm(tifs):  # Loop over all Sentinel-1 images
            for band in range(1, 4):  # Loop over all three bands (indexed 1 to 3)
                rasterstatsmulti = RasterstatsMultiProc(df=df, tif=tif, all_touched=ALL_TOUCHED)

                if MULTI_PROC_ZONAL_STATS:
                    results_df = rasterstatsmulti.calc_zonal_stats_multiproc()     
                else:
                    results_df = rasterstatsmulti.calc_zonal_stats(band=band, prog_bar=False) 

                del rasterstatsmulti

                stats_cols = {
                    'min': tif.stem + '_B' + str(band) + '_min',
                    'max': tif.stem + '_B' + str(band) + '_max',
                    'mean': tif.stem + '_B' + str(band) + '_mean',
                    'std': tif.stem + '_B' + str(band) + '_std',
                    'median': tif.stem + '_B' + str(band) + '_median',
                }

                results_df = results_df.rename(columns=stats_cols)

                # Note: The * operator iterates through the list (https://stackoverflow.com/a/56736691/12045808)
                df_potato_stats[df_name] = df_potato_stats[df_name].merge(results_df[['id', *stats_cols.values()]], left_on='id', right_on='id')

        if not shp_path.parent.exists():
            os.makedirs(shp_path.parent)

        # Set the CRS in the geodataframe to be wkt format (otherwise you won't be able to save as a shapefile)
        #df_potato_stats[df_name].crs = df_potato_stats[df_name].crs['init'].to_wkt()
        #df_potato_stats[df_name] = df_potato_stats[df_name].dropna()
        df_potato_stats[df_name].to_pickle(pkl_path) 
'''

'\ntifs = sorted((PROJ_PATH / \'data\' / \'raw\' / \'Sentinel-1\').glob(\'*.tif\'))\ndf_potato_stats = df_potato.copy()\n\nfor df_name, df in df_potato.items(): # Loop over all field polygon years\n    pkl_name = df_name + \'_stats\' \n    pkl_path = (PROJ_PATH / \'data\' / \'processed\' / pkl_name).with_suffix(\'.pkl\')\n    if pkl_path.exists():\n        print("Zonal statistics have already been calculated for: " + df_name)\n    else:\n        print("Calculating zonal statistics for: " + df_name)\n        #df = df.head(20)  # For debugging to (ie. only process 20 fields)\n        for tif in tqdm(tifs):  # Loop over all Sentinel-1 images\n            for band in range(1, 4):  # Loop over all three bands (indexed 1 to 3)\n                rasterstatsmulti = RasterstatsMultiProc(df=df, tif=tif, all_touched=ALL_TOUCHED)\n\n                if MULTI_PROC_ZONAL_STATS:\n                    results_df = rasterstatsmulti.calc_zonal_stats_multiproc()     \n                else:\n                

time: 16.9 ms


In [10]:
!pip install h5netcdf

time: 3.12 s


In [77]:
# We now want to create an xarray dataset based on the dataframe, with the zonal statistics as extra dimensions
tifs = sorted((PROJ_PATH / 'data' / 'raw' / 'Sentinel-1').glob('*.tif'))
df_potato_stats = df_potato.copy()

for df_name, df in df_potato.items(): # Loop over all field polygon years
    netcdf_name = df_name + '_stats' 
    netcdf_path = (PROJ_PATH / 'data' / 'processed' / netcdf_name).with_suffix('.nc')
    #if netcdf_path.exists():
    if not '2019' in df_name:
        print("Zonal statistics have already been calculated for: " + df_name)
    else:
        print("Calculating zonal statistics for: " + df_name)
        ### FOR DEBUGGING ###
        df = df.head(200)  
        #tifs = tifs[0:3]
        #####################
        
        # Load the dataframe into xarray 
        ds = xr.Dataset.from_dataframe(df.set_index('id'))  # Use field_id (named 'id') as index
        ds = ds.rename({'id': 'field_id'})  
        ds = ds.drop('geometry')  # Cannot be saved to netcdf format

        # Find the dates of all the tif files and assign them as new coordinates
        dates_str = list(map(lambda x: x.stem[4:12], tifs))
        dates = pd.to_datetime(dates_str)
        ds = ds.assign_coords({'date': dates})
        
        # Assign polarization coordinates
        ds = ds.assign_coords({'polarization': ['VH', 'VV', 'VV-VH']})

        # Create the empty arrays for the xarray data_vars
        num_fields = ds.dims['field_id']
        num_dates = len(dates)
        num_polarizations = ds.dims['polarization']
        stats_min_array = np.zeros((num_fields, num_dates, num_polarizations), dtype=np.float32)  
        stats_max_array = np.zeros_like(stats_min_array)
        stats_mean_array = np.zeros_like(stats_min_array)
        stats_std_array = np.zeros_like(stats_min_array)
        stats_median_array = np.zeros_like(stats_min_array)
        satellite_array = [None] * num_dates
        pass_mode_array = [None] * num_dates
        relative_orbit_array = np.zeros((num_dates), dtype=np.int16)
        
        # Calculate the zonal stats
        for date_index, tif in enumerate(tqdm(tifs)):  # Loop over all Sentinel-1 images
            # Get metadata for satellite pass from the filename of the .tif file
            satellite = tif.stem[0:3]
            pass_mode = tif.stem[20:23]
            relative_orbit = tif.stem[24:27]
            
            # Perform zonal statistics 
            for band in range(1, 4):  # Loop over all polarizations, including cross-polarization (indexed 1 to 3)
                rasterstatsmulti = RasterstatsMultiProc(df=df, tif=tif, all_touched=ALL_TOUCHED)

                if MULTI_PROC_ZONAL_STATS:
                    results_df = rasterstatsmulti.calc_zonal_stats_multiproc()     
                else:
                    results_df = rasterstatsmulti.calc_zonal_stats(band=band, prog_bar=False) 

                del rasterstatsmulti
                
                # Check if the ordering of the field_ids are the same in the xarray dataset and the results_df
                # (they must be - otherwise the calculated statistics will be assigned to the wrong elements in the statistics arrays)
                for i in np.random.randint(low=0, high=num_fields, size=20):
                    ds_field_id = ds.isel(field_id=i)['field_id'].values
                    df_field_id = results_df.iloc[i]['id']
                    assert ds_field_id == df_field_id 
                
                # Update the arrays
                polarization_index = band-1  # Get the indexing right
                stats_min_array[:, date_index, polarization_index] = results_df['min']
                stats_max_array[:, date_index, polarization_index] = results_df['max']
                stats_mean_array[:, date_index, polarization_index] = results_df['mean']
                stats_std_array[:, date_index, polarization_index] = results_df['std']
                stats_median_array[:, date_index, polarization_index] = results_df['median']
                satellite_array[date_index] = satellite
                pass_mode_array[date_index] = pass_mode 
                relative_orbit_array[date_index] = relative_orbit 
                
        # Load the arrays into xarray
        ds['stats_min']=(['field_id', 'date', 'polarization'], stats_min_array)
        ds['stats_max']=(['field_id', 'date', 'polarization'], stats_max_array)
        ds['stats_mean']=(['field_id', 'date', 'polarization'], stats_mean_array)
        ds['stats_std']=(['field_id', 'date', 'polarization'], stats_std_array)
        ds['stats_median']=(['field_id', 'date', 'polarization'], stats_median_array)
        ds['satellite']=(['date'], satellite_array)
        ds['pass_mode']=(['date'], pass_mode_array)
        ds['relative_orbit']=(['date'], relative_orbit_array)
        
        # Use proper dtypes in the datset to save space and memory
        ds['field_id'] = ds['field_id'].astype(np.int32) 
        ds['afgkode'] = ds['afgkode'].astype(np.int16) 
        ds['gb'] = ds['gb'].astype(np.float32) 
        ds['gbanmeldt'] = ds['gbanmeldt'].astype(np.float32) 
        ds['imk_areal'] = ds['imk_areal'].astype(np.float32) 

        # Save the dataset
        if not netcdf_path.parent.exists():
            os.makedirs(netcdf_path.parent)
        ds = ds.sortby('date')  # Sort the dates (they are scrambled due to naming of the tif files starting with 'S1A' and 'S1B')
        ds.to_netcdf(netcdf_path, engine='h5netcdf')

Zonal statistics have already been calculated for: FieldPolygons2017
Zonal statistics have already been calculated for: FieldPolygons2018
Calculating zonal statistics for: FieldPolygons2019



time: 3min 48s


In [95]:
netcdf_path = (PROJ_PATH / 'data' / 'processed' / 'FieldPolygons2019_stats').with_suffix('.nc')
with xr.open_dataset(netcdf_path, engine="h5netcdf") as ds:
    ds = ds.isel(field_id=[1, 2, 3])  # Only select one field or the plotting fucks up
    ds = ds.sel(date=slice('2019-01-01', '2019-11-24'))
    #ds = ds.sel(polarization='VV')
    #ds = ds.sel(satellite='S1A')
    
    # Format the dataframe to work well with Seaborn for plotting
    df = ds.to_dataframe()
    df = df.reset_index()  # Remove MultiIndex
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df = df.drop(columns=['cvr', 'gb', 'gbanmeldt', 'journalnr', 'marknr'])

    # Perhaps it would be an idea to have field centroids (as lat, lon) to find fields within geographic area
    
df

,date,field_id,polarization,afgkode,afgroede,imk_areal,pass_mode,relative_orbit,satellite,stats_max,stats_mean,stats_median,stats_min,stats_std
0,2019-01-04,61952339,VH,151,"Kartofler, stivelses-",6.02,DSC,139,S1B,-12.0,-18.595764,-19.0,-28.0,2.405783
1,2019-01-04,61952339,VV,151,"Kartofler, stivelses-",6.02,DSC,139,S1B,-4.0,-10.579189,-11.0,-17.0,2.248529
2,2019-01-04,61952339,VV-VH,151,"Kartofler, stivelses-",6.02,DSC,139,S1B,16.0,7.521179,8.0,-1.0,2.656347
3,2019-01-04,61952356,VH,150,"Kartofler, lægge- (egen opformering)",5.49,DSC,139,S1B,-12.0,-20.370968,-20.0,-29.0,2.455139
4,2019-01-04,61952356,VV,150,"Kartofler, lægge- (egen opformering)",5.49,DSC,139,S1B,-7.0,-11.939516,-12.0,-18.0,1.811804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,2019-10-31,61952356,VV,150,"Kartofler, lægge- (egen opformering)",5.49,DSC,139,S1B,-4.0,-10.273279,-10.0,-17.0,1.952424
446,2019-10-31,61952356,VV-VH,150,"Kartofler, lægge- (egen opformering)",5.49,DSC,139,S1B,26.0,9.147774,9.0,0.0,2.979421
447,2019-10-31,61952436,VH,151,"Kartofler, stivelses-",1.84,DSC,139,S1B,-9.0,-14.822823,-15.0,-20.0,1.870063
448,2019-10-31,61952436,VV,151,"Kartofler, stivelses-",1.84,DSC,139,S1B,-2.0,-7.480481,-7.0,-13.0,1.999342


time: 110 ms


In [83]:
#ds = xr.open_dataset(netcdf_path, engine="h5netcdf")
#ds = ds.drop_sel(satellite=['S1A'])
#ds  # Remember to close the dataset before the netcdf file can be rewritten in cells above

<xarray.Dataset>
Dimensions:         (date: 50)
Coordinates:
  * date            (date) datetime64[ns] 2019-01-04 2019-01-10 ... 2019-10-31
    field_id        int32 61952361
    polarization    <U2 'VV'
Data variables:
    afgkode         (date) float64 151.0 151.0 151.0 151.0 ... 151.0 151.0 151.0
    afgroede        (date) object 'Kartofler, stivelses-' ... 'Kartofler, stivelses-'
    cvr             (date) object '28496257' '28496257' ... '28496257'
    gb              (date) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    gbanmeldt       (date) float64 4.88 4.88 4.88 4.88 ... 4.88 4.88 4.88 4.88
    imk_areal       (date) float64 4.88 4.88 4.88 4.88 ... 4.88 4.88 4.88 4.88
    journalnr       (date) object '19-0017518' '19-0017518' ... '19-0017518'
    marknr          (date) object '31-2' '31-2' '31-2' ... '31-2' '31-2' '31-2'
    pass_mode       (date) object 'DSC' 'DSC' 'DSC' 'DSC' ... 'DSC' 'DSC' 'DSC'
    relative_orbit  (date) float64 139.0 139.0 139.0 139.0 ... 139.0 139.0 139.0
    satellite       (date) object 'S1B' 'S1A' 'S1B' 'S1A' ... 'S1B' 'S1A' 'S1B'
    stats_max       (date) float32 -7.0 -8.0 -7.0 -8.0 ... -3.0 -3.0 -3.0 -3.0
    stats_mean      (date) float32 -11.720137 -14.357142 ... -8.906713 -9.150685
    stats_median    (date) float32 -12.0 -14.0 -11.0 -15.0 ... -9.0 -9.0 -9.0
    stats_min       (date) float32 -18.0 -25.0 -19.0 -23.0 ... -15.0 -14.0 -15.0
    stats_std       (date) float32 1.7412722 2.403347 ... 1.7906184 1.7634381

time: 65.3 ms
